## Dataset Split

In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
import pandas as pd
import re

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
#!wget http://download.cs.stanford.edu/deep/CheXpert-v1.0-small.zip


In [11]:
load_dotenv(find_dotenv())
chestxray14_folder = Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY"))
chexpert_folder = Path(os.environ.get("CHEXPERT_FULL_PREPROCESSED_DATASET_DIRECTORY"))
print(chestxray14_folder)
print(chexpert_folder)

data/chestxray14/scale_256
data/chexpert/preprocessed-256-crop


In [12]:
chestxray14_labels = pd.read_csv(chestxray14_folder / "meta/data/labels.csv")
chexpert_train_labels = pd.read_csv(chexpert_folder / "train.csv")
#chexpert_test_labels = pd.read_csv(chexpert_folder / "test.csv")

In [13]:
chestxray14_labels.head()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage_Width,OriginalImage_Height,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,...,Infiltration,Consolidation,Fibrosis,No Finding,Effusion,Nodule,Mass,Hernia,Emphysema,Pneumothorax
0,00000001_000.png,0,1,57,M,PA,2682,2749,0.143,0.143,...,False,False,False,False,False,False,False,False,False,False
1,00000001_001.png,1,1,58,M,PA,2894,2729,0.143,0.143,...,False,False,False,False,False,False,False,False,True,False
2,00000001_002.png,2,1,58,M,PA,2500,2048,0.168,0.168,...,False,False,False,False,True,False,False,False,False,False
3,00000002_000.png,0,2,80,M,PA,2500,2048,0.171,0.171,...,False,False,False,True,False,False,False,False,False,False
4,00000003_001.png,0,3,74,F,PA,2500,2048,0.168,0.168,...,False,False,False,False,False,False,False,True,False,False


In [14]:
chexpert_train_labels.head()

,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0,1
1,1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,...,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN,2
2,2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,...,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
3,3,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,...,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
4,4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,3


In [9]:
chexpert_test_labels.head()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,CheXpert-v1.0-small/valid/patient64541/study1/...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64541
1,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64542
2,CheXpert-v1.0-small/valid/patient64542/study1/...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64542
3,CheXpert-v1.0-small/valid/patient64543/study1/...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64543
4,CheXpert-v1.0-small/valid/patient64544/study1/...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64544


In [8]:
!ls data/chexpert/full/CheXpert-v1.0-small/train/patient28746/

ls: data/chexpert/full/CheXpert-v1.0-small/train/patient28746/: No such file or directory


In [28]:
def get_patient_id (row):
    patient_folder = re.findall(r"patient[0-9]+", row["Path"])
    if len(patient_folder) > 0:
        patient_number = re.findall(r"[0-9]+", patient_folder[0])
    if len(patient_number) > 0:
        return (patient_number[0])

In [29]:
chexpert_train_labels["patient_id"] = chexpert_train_labels.apply(get_patient_id, axis = 1)
chexpert_test_labels["patient_id"] = chexpert_test_labels.apply(get_patient_id, axis = 1)


In [33]:
chexpert_train_labels["patient_id"].value_counts()

28746    92
33155    92
04462    89
24163    86
34615    85
         ..
58723     1
05100     1
41507     1
63492     1
07717     1
Name: patient_id, Length: 64540, dtype: int64

In [34]:
chexpert_test_labels["patient_id"].value_counts()

64616    3
64547    3
64581    3
64595    2
64560    2
        ..
64694    1
64661    1
64692    1
64730    1
64669    1
Name: patient_id, Length: 200, dtype: int64

In [88]:
#!mkdir ~/idp-radio-1/data/dev_dataset
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small
#!mkdir ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train
!ls ~/idp-radio-1/data/dev_dataset/CheXpert-v1.0-small/train

In [70]:
#chexpert_train_labels.to_csv(chexpert_folder / "train.csv")
#chexpert_test_labels.to_csv(chexpert_folder / "test.csv")

In [10]:
chexpert_train_labels.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices', 'patient_id'],
      dtype='object')

In [5]:
no_finding_mask_train = chexpert_train_labels["No Finding"] == 1.0
#no_finding_mask_test = chexpert_test_labels["No Finding"] == 1.0

In [6]:
finding_columns = ['Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture']
replacement_values = {finding: 0.0 for finding in finding_columns}

chexpert_train_labels[no_finding_mask_train] = chexpert_train_labels[no_finding_mask_train].fillna(value=replacement_values)
#chexpert_test_labels[no_finding_mask_test] = chexpert_test_labels[no_finding_mask_test].fillna(value=replacement_values)

In [7]:
chexpert_train_labels["No Finding"].value_counts()

1.0    22381
Name: No Finding, dtype: int64

In [8]:
no_finding_mask_train.value_counts()

False    201033
True      22381
Name: No Finding, dtype: int64

In [9]:
chexpert_train_labels[no_finding_mask_train]

,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,patient_id
0,0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
5,5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4
6,6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4
7,7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
8,8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223388,223388,CheXpert-v1.0-small/train/patient64519/study1/...,Female,33,Frontal,AP,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64519
223389,223389,CheXpert-v1.0-small/train/patient64520/study1/...,Female,65,Frontal,AP,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,64520
223391,223391,CheXpert-v1.0-small/train/patient64522/study1/...,Female,21,Frontal,AP,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64522
223402,223402,CheXpert-v1.0-small/train/patient64532/study1/...,Female,52,Frontal,AP,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64532


In [10]:
chexpert_train_labels.to_csv(chexpert_folder / "nofinding_train.csv")
#chexpert_test_labels.to_csv(chexpert_folder / "nofinding_test.csv")